In [ ]:
import os
import pandas as pd
import numpy as np

from quick_pp.objects import Project

####  Steps to create project
1. Run the next cell.
2. Specify the example either clastic or carbonate.
3. MOCK.qppp project will be saved in notebooks\data\04_project folder.

* Note that the required curves in the LAS files are 'GR', 'RT', 'NPHI', 'RHOB'

In [ ]:
data_path = 'data/'
filenames = []
for root, dirs, files in os.walk(data_path):
    for file in files:
        filenames.append(os.path.join(root, file)) if file.endswith('.las') and not 'seismic' in root  else None
print(filenames)

project_name = "VOLVE"
project = Project(name=project_name)
project.read_las(filenames)
# clear_output()               

In [ ]:
# Process core data
data = project.get_all_data()
remove_rows = (
    (data.WELL_NAME == '15-9-19-SR') & ((data.DEPTH < 560) | (data.DEPTH > 4590))
)
merged_df = data[~remove_rows].copy()

merged_core_df = pd.DataFrame()
for well_name, well_data in data.groupby('WELL_NAME'):
    if well_name in ['15-9-19-A', '15-9-19-BT2']:
        core_data = pd.read_excel(rf"data\{well_name}_RCA.xls", skiprows=3).iloc[1:]
        core_data['WELL_NAME'] = well_name
        core_data['DEPTH'] = pd.to_numeric(core_data['Depth'], errors='coerce')
        core_data['CPORE'] = pd.to_numeric(core_data['Por., hor.'], errors='coerce') / 100
        core_data['CPERM'] = pd.to_numeric(core_data['Kl, hor'], errors='coerce')
        merged_core_df = pd.concat([merged_core_df, core_data])

        # Merge with well logs
        temp_df = pd.merge_asof(well_data, core_data[['DEPTH', 'CPORE', 'CPERM']],
                                direction='nearest', on='DEPTH', tolerance=0.1524 / 2)
        merged_df.loc[merged_df.WELL_NAME == well_name, ['CPORE', 'CPERM']] = temp_df[['CPORE', 'CPERM']]

merged_core_df.to_csv(r'data\merged_core_data.csv')

In [ ]:
# Process facies data
merged_facies_df = pd.DataFrame()
for well_name, well_data in merged_df.groupby('WELL_NAME'):
    if well_name in ['15-9-19-A', '15-9-19-BT2', '15-9-19-SR']:
        facies_df = pd.read_excel(rf"data\{well_name}_Facies.xlsx")
        facies_df['WELL_NAME'] = well_name
        facies_df['TOP_DEPTH'] = pd.to_numeric(facies_df['* Top Depth (meters)'], errors='coerce')
        facies_df['BASE_DEPTH'] = pd.to_numeric(facies_df['* Base Depth (meters)'], errors='coerce')
        facies_df['LITHOFACIES'] = facies_df['Litho Class']
        merged_facies_df = pd.concat([merged_facies_df, facies_df])

expanded_rows = []
for index, row in merged_facies_df.iterrows():
    # Define the depth range for the current row. 
    # np.arange is great for this, especially with floating point steps.
    # We add a small epsilon to the stop value to make sure it's inclusive.
    depth_range = np.arange(row['TOP_DEPTH'], row['BASE_DEPTH'] + 0.5, 0.5)
    
    for depth in depth_range:
        expanded_rows.append({
            'WELL_NAME': row['WELL_NAME'],
            'DEPTH': depth,
            'LITHOFACIES': row['LITHOFACIES']
        })
merged_facies_df = pd.DataFrame(expanded_rows).drop_duplicates(['WELL_NAME', 'DEPTH', 'LITHOFACIES'])
merged_facies_df = merged_facies_df.replace({
    'UNKNOWN': np.nan, 'UNDEFINED': np.nan
})
merged_facies_df.to_csv(r'data\merged_facies_data.csv')

In [ ]:
# Process zones
tops_df = pd.read_excel(rf"data\strat_litho_wellbore.xlsx")
tops_df['WELL_NAME'] = tops_df['Wellbore name'].str.replace('/', '-').str.replace(' ', '-')
for well_name, well_data in merged_df.groupby('WELL_NAME'):
    marker_df = tops_df[tops_df.WELL_NAME == well_name].copy()
    marker_df['DEPTH'] = marker_df['Top depth [m]'].astype('float')
    marker_df['ZONES'] = marker_df['Lithostrat. unit']

    # Merge with well logs
    temp_df = pd.merge_asof(well_data, marker_df[['DEPTH', 'ZONES']],
                            direction='nearest', on='DEPTH', tolerance=0.1524 / 2)
    merged_df.loc[merged_df.WELL_NAME == well_name, ['ZONES']] = temp_df[['ZONES']]
merged_df['ZONES'] = merged_df.ZONES.ffill()

In [ ]:
# Process TVDSS
import wellpathpy as wpp

well_survey = pd.read_csv(r"data\VOLVE_15-9_Surveys.csv")[
    ['MD (m RKB)', 'Inc (deg)', 'Azim (deg)', 'UTM E/W (m)', 'UTM N/S (m)', 'TVD (m RKB)', 'WELL_NAME']
]
well_survey.columns = ['md', 'incl', 'azim', 'x', 'y', 'z', 'WELL_NAME']
for well_name, well_data in merged_df.groupby('WELL_NAME'):
    coords = well_survey[well_survey.WELL_NAME == well_name]
    dev_survey = wpp.deviation(coords['md'], coords['incl'], coords['azim'])
    tvd = dev_survey.minimum_curvature().resample(well_data.DEPTH.values).depth
    mask = (merged_df.WELL_NAME == well_name)

    print(well_name, well_data.shape, len(tvd))
    print(sum(mask))

    merged_df.loc[mask, 'TVD'] = tvd

In [ ]:
# Merge with well logs
for well_name, well_data in merged_df.groupby('WELL_NAME'):
    facies_df = merged_facies_df[merged_facies_df.WELL_NAME == well_name].sort_values('DEPTH').drop(columns='WELL_NAME')
    well_data = well_data.sort_values('DEPTH')
    temp_df = pd.merge_asof(well_data, facies_df,
                            direction='nearest', on='DEPTH', tolerance=0.1524 / 2)
    merged_df.loc[merged_df.WELL_NAME == well_name, 'LITHOFACIES'] = temp_df['LITHOFACIES']

# Save the update data to the project
project.update_data(merged_df)
project.save()